In [1]:
import os
os.environ["PYOPENGL_PLATFORM"] = "egl"

In [2]:
import sys
import ast
import mysql.connector
import numpy as np
import pandas as pd
import torch
import hdbscan
from torch import nn
from torchvision import models, transforms
import matplotlib.pyplot as plt
from sklearn.preprocessing import StandardScaler
from stl import mesh as np_mesh  # Import numpy-stl
from scipy.spatial.distance import cdist

from joblib import Parallel, delayed
import gc
import psutil
import shutil
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.metrics import silhouette_score, calinski_harabasz_score, davies_bouldin_score

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)

cpu


In [3]:
# Paths
input_folder = r"C:\Users\soura\Desktop\PDSV_codes\stliitm"  # Path to your STL files

if os.path.exists(input_folder):
    print("Directory exists")
else:
    print("Directory does not exist")


Directory exists


In [4]:
base_path = r"C:\Users\soura\Desktop\PDSV_codes\mtech_project_feature_folder"
print("Base path:", base_path)

proj_features_folder = os.path.join(base_path, 'proj_features')   # Folder to save projection features
os.makedirs(proj_features_folder, exist_ok=True)
print("feature folder:", proj_features_folder)


normal_features_folder = os.path.join(base_path, 'normal_features')   # Folder to save normal features
os.makedirs(normal_features_folder, exist_ok=True)
print("feature folder:", normal_features_folder)

surface_features_folder = os.path.join(base_path, 'surface_features')   # Folder to save normal features
os.makedirs(surface_features_folder, exist_ok=True)
print("feature folder:", surface_features_folder)

Base path: C:\Users\soura\Desktop\PDSV_codes\mtech_project_feature_folder
feature folder: C:\Users\soura\Desktop\PDSV_codes\mtech_project_feature_folder\proj_features
feature folder: C:\Users\soura\Desktop\PDSV_codes\mtech_project_feature_folder\normal_features
feature folder: C:\Users\soura\Desktop\PDSV_codes\mtech_project_feature_folder\surface_features


In [5]:
# Extract features and perform clustering.
stl_files = [f for f in os.listdir(input_folder) if f.endswith('.stl')]  # listing all stl files from the input file.
print(stl_files)
print(len(stl_files))


['1013826_1-323326_prt.stl', '1013858_1-643172_prt.stl', '1038615_116-1054_prt.stl', '1040428_0000000317p2_prt.stl', '1040431_0000000209_prt.stl', '1040433_p2_prt.stl', '1040434_0000000309helx_prt.stl', '1040435_0000000313hole_prt.stl', '1040436_0000000305p_prt.stl', '1040437_0000000306p1_prt.stl', '1040439_0000000308helx_prt.stl', '1040440_0000000312_prt.stl', '1040777_130-4534_prt.stl', '1040867_353-781fr_prt.stl', '1041040_0000000315p1_prt.stl', '1041074_01-201-4520_mir_prt.stl', '1041132_104-4993fr_mir_prt.stl', '1041202_237-80_mir_prt.stl', '1041235_01-201-3290fr_mir_prt.stl', '1041315_103-2616-a_mir_prt.stl', '1041354_0000000348_prt.stl', '1041400_237-42_mir_prt.stl', '1041401_1-513840fr_mir_prt.stl', '1041506_104-4836fr_mir_prt.stl', '1041594_0000000127_prt.stl', '1041595_0000000123_prt.stl', '1041596_0000000128_prt.stl', '1041597_0000000126_prt.stl', '1045086_100-9634fr_prt.stl', '1045305_0000000357_prt.stl', '1131335_1180-1530_prt.stl', '1131337_191-12933_prt.stl', '1131339_19

In [6]:
class Proj_DBhelper:
    def __init__(self, data_base):
        try:
            self.conn = mysql.connector.connect(host="localhost", user="root", password="", database=data_base)
            self.mycursor = self.conn.cursor()
        except:
            print("some has occured")
            sys.exit(0)
        else:
            print("Database is connected")

    def create_table(self, table_name):
        try:
            create_table_query = f"""
                                    CREATE TABLE IF NOT EXISTS {table_name} (
                                        id INT(11) AUTO_INCREMENT PRIMARY KEY,
                                        file_name VARCHAR(255) NULL,
                                        proj_feature_vector VARCHAR(255) NULL,
                                        cluster_label INT(11) NULL
                                    );"""
            self.mycursor.execute(create_table_query)
            self.conn.commit()
        except:
            print("Table has not been created")
            sys.exit(0)
        else:
            print("Table is ready.")

    def register(self, stl_file, cluster_label = -1):
        try:
            insert_query = """INSERT INTO `stl_5000` (`file_name`, `cluster_label`) VALUES (%s, %s);"""
            self.mycursor.execute(insert_query, (stl_file, cluster_label))
            self.conn.commit()
        except:
            print(f"{stl_file} could not entered in the database")
        else:
            print(f"{stl_file} is entered in the database")
        
        
    def update_cluster_label(self, stl_file, cluster_label):
        try:
            update_query = """UPDATE `stl_5000` SET `cluster_label` = %s WHERE `stl_5000`.`file_name` = %s;"""
            self.mycursor.execute(update_query, (cluster_label, stl_file))
            self.conn.commit()
        except:
            print(f"could not able to update for the {stl_file}")
        else:
            print(f"updatation of label is sucessfull for the {stl_file}")
        
        
    def update_feature_vector(self, stl_file, proj_feature):
        try:
            update_query = """UPDATE `stl_5000` SET `proj_feature_vector` = %s WHERE `stl_5000`.`file_name` = %s;"""
            self.mycursor.execute(update_query, (proj_feature, stl_file))
            self.conn.commit()
        except:
            print(f"The vectors could not enter in the database")
        else:
            print(f"The vectors are entered in the database")
        
    def search_feature_vector(self, stl_file):
        try:
            self.mycursor.execute("""SELECT proj_feature_vector FROM stl_5000
                                WHERE file_name LIKE '{}'""".format(stl_file))
            data = self.mycursor.fetchone()
            return data[0]
        except:
            return None
        
    def search_feature_vector_through_id(self, id_number):
        try:
            self.mycursor.execute("""SELECT file_name, proj_feature_vector FROM stl_5000
                                WHERE id LIKE '{}'""".format(id_number))
            data = self.mycursor.fetchone()
            return data[0], data[1]
        except:
            return None, None
        
    def search_cluster_label_files(self, cluster_label):
        try:
            search_query = """SELECT `file_name` FROM `stl_5000` WHERE `cluster_label` LIKE %s"""
            self.mycursor.execute(search_query, (cluster_label,))
            data = self.mycursor.fetchall()
            return data, True
        except:
            return None, False
    
    def search_corresponding_label(self, stl_file):
        try:
            search_query = """SELECT `cluster_label` FROM `stl_5000` WHERE `file_name` LIKE %s"""
            self.mycursor.execute(search_query, (stl_file,))
            data = self.mycursor.fetchone()
            return data[0]
        except:
            return None
        
    def search_max_cluster_label(self):
        try:
            query = """SELECT MAX(cluster_label) FROM stl_5000"""
            self.mycursor.execute(query)
            max_value = self.mycursor.fetchone()
            return max_value[0]
        except:
            return -1
        
    def search_total_files(self):
        try:
            query = """SELECT MAX(id) FROM stl_5000"""
            self.mycursor.execute(query)
            max_value = self.mycursor.fetchone()
            return max_value[0]
        except:
            return 0
    

In [7]:
class Proj_Feature_Data_Base:
    def __init__(self, data_base):                               
        self.db = Proj_DBhelper(data_base)   #connect to the database

    def table_creation(self, table_name):
        self.db.create_table(table_name)    #create table
        
    def send_files_to_db(self, stl_files):
        for file in stl_files:
            self.db.register(file)
                
    def send_labels_to_db(self, stl_file, cluster_label):
        self.db.update_cluster_label(stl_file, cluster_label)

    def send_feature_vector_to_db(self, stl_file, proj_vector):
        self.db.update_feature_vector(stl_file, proj_vector)
        
    def find_feature_vector(self, stl_file):
        vector_file_name = self.db.search_feature_vector(stl_file)
        if vector_file_name:
            return vector_file_name, True
        else:
            return None, False
        
    def find_feature_vector_through_id(self, id_number):
        file_name, vector_file_name = self.db.search_feature_vector_through_id(id_number)
        if vector_file_name:
            return file_name, vector_file_name, True
        else:
            return file_name, None, False
        
    def find_files(self, cluster_label):
        data, flag = self.db.search_cluster_label_files(cluster_label)
        if(flag):
            if(len(data) > 0):
                return data
        return None

    def find_label(self, stl_file):
        data = self.db.search_corresponding_label(stl_file)
        if data:
            return data
        else:
            return None
    
    def maximum_cluster_label(self):
        number = self.db.search_max_cluster_label()
        return number
    
    def count_files(self):
        number = self.db.search_total_files()
        return number

    def data_distribution(self):
        length = self.maximum_cluster_label()
        dictionary = {}
        for i in range(length+1):
            num_files = self.db.search_cluster_label_files(i)
            dictionary[i] = len(num_files)

        clusters = list(dictionary.keys())
        values = list(dictionary.values())
        
        fig = plt.figure(figsize = (20, 10))

        # creating the bar plot
        plt.bar(clusters, values, color ='maroon', width = 0.8)

        plt.xlabel("Cluster labels")
        plt.ylabel("No. of files of corresponding labels")
        plt.title("clusters vs no. of files")
        plt.show()

In [8]:
s1 = Proj_Feature_Data_Base("mtech_project_database")

Database is connected


In [9]:
s1.table_creation("stl_5000")

Table is ready.


In [20]:
num_files = s1.count_files()
print(num_files)

4278


In [22]:
s1.find_feature_vector_through_id(4266)

('225132_testpart_3_prt.stl',
 'C:\\Users\\soura\\Desktop\\PDSV_codes\\stl_5000_feature_folder\\proj_features\\225132_testpart_3_prt_features.npy',
 True)

In [10]:
s1.send_files_to_db(stl_files)

1013826_1-323326_prt.stl is entered in the database
1013858_1-643172_prt.stl is entered in the database
1038615_116-1054_prt.stl is entered in the database
1040428_0000000317p2_prt.stl is entered in the database
1040431_0000000209_prt.stl is entered in the database
1040433_p2_prt.stl is entered in the database
1040434_0000000309helx_prt.stl is entered in the database
1040435_0000000313hole_prt.stl is entered in the database
1040436_0000000305p_prt.stl is entered in the database
1040437_0000000306p1_prt.stl is entered in the database
1040439_0000000308helx_prt.stl is entered in the database
1040440_0000000312_prt.stl is entered in the database
1040777_130-4534_prt.stl is entered in the database
1040867_353-781fr_prt.stl is entered in the database
1041040_0000000315p1_prt.stl is entered in the database
1041074_01-201-4520_mir_prt.stl is entered in the database
1041132_104-4993fr_mir_prt.stl is entered in the database
1041202_237-80_mir_prt.stl is entered in the database
1041235_01-201-32

In [11]:
class Normal_DBhelper:
    def __init__(self, data_base):
        try:
            self.conn = mysql.connector.connect(host="localhost", user="root", password="", database=data_base)
            self.mycursor = self.conn.cursor()
        except:
            print("some has occured")
            sys.exit(0)
        else:
            print("Database is connected")

    def create_table(self, table_name):
        try:
            create_table_query = f"""
                                    CREATE TABLE IF NOT EXISTS {table_name} (
                                        id INT(11) AUTO_INCREMENT PRIMARY KEY,
                                        file_name VARCHAR(255) NULL,
                                        normal_feature_vector VARCHAR(255) NULL,
                                        surface_feature_vector VARCHAR(255) NULL
                                    );"""
            self.mycursor.execute(create_table_query)
            self.conn.commit()
        except:
            print("Table could not be created")
            sys.exit(0)
        else:
            print("Table is ready.")

    def register(self, stl_file):
        try:
            insert_query = """INSERT INTO `stl_5000_normal` (`file_name`) VALUES (%s);"""
            self.mycursor.execute(insert_query, (stl_file,))
            self.conn.commit()
        except:
            print(f"{stl_file} could not entered in the database")
        else:
            print(f"{stl_file} has entered in the database")
        
        
    def update_feature_vector(self, stl_file, normal_feature, surface_feature):
        try:
            update_query = """UPDATE `stl_5000_normal` SET `normal_feature_vector` = %s, `surface_feature_vector` = %s WHERE `stl_5000_normal`.`file_name` = %s;"""
            self.mycursor.execute(update_query, (normal_feature, surface_feature, stl_file))
            self.conn.commit()
        except:
            print(f"The vectors could not enter in the database")
        else:
            print(f"The vectors have been entered in the database")
        
    def search_feature_vector(self, stl_file):
        try:
            self.mycursor.execute("""SELECT normal_feature_vector, surface_feature_vector FROM stl_5000_normal
                                WHERE file_name LIKE '{}'""".format(stl_file))
            data = self.mycursor.fetchone()
            return data[0], data[1]
        except:
            return None, None


In [12]:
class Normal_Surface_Database:
    def __init__(self, data_base):                               
        self.db = Normal_DBhelper(data_base)   #connect to the database

    def table_creation(self, table_name):
        self.db.create_table(table_name)    #create table
        
    def send_files_to_db(self, stl_files):
        for file in stl_files:
            self.db.register(file)

    def send_feature_vector_to_db(self, stl_file, normal_vector, surface_vector):
        self.db.update_feature_vector(stl_file, normal_vector, surface_vector)
        
    def find_feature_vector(self, stl_file):
        normal_vector_file_path, surface_vector_file_path = self.db.search_feature_vector(stl_file)
        if normal_vector_file_path:
            return normal_vector_file_path, surface_vector_file_path,True
        else:
            return None, None, False

In [13]:
n1 = Normal_Surface_Database("mtech_project_database")

Database is connected


In [14]:
n1.table_creation("stl_5000_normal")

Table is ready.


In [15]:
n1.send_files_to_db(stl_files)

1013826_1-323326_prt.stl has entered in the database
1013858_1-643172_prt.stl has entered in the database
1038615_116-1054_prt.stl has entered in the database
1040428_0000000317p2_prt.stl has entered in the database
1040431_0000000209_prt.stl has entered in the database
1040433_p2_prt.stl has entered in the database
1040434_0000000309helx_prt.stl has entered in the database
1040435_0000000313hole_prt.stl has entered in the database
1040436_0000000305p_prt.stl has entered in the database
1040437_0000000306p1_prt.stl has entered in the database
1040439_0000000308helx_prt.stl has entered in the database
1040440_0000000312_prt.stl has entered in the database
1040777_130-4534_prt.stl has entered in the database
1040867_353-781fr_prt.stl has entered in the database
1041040_0000000315p1_prt.stl has entered in the database
1041074_01-201-4520_mir_prt.stl has entered in the database
1041132_104-4993fr_mir_prt.stl has entered in the database
1041202_237-80_mir_prt.stl has entered in the database

In [16]:
# Load pre-trained SqueezeNet and modify it
class FeatureExtractor(nn.Module):
    def __init__(self):
        super(FeatureExtractor, self).__init__()
        self.squeezenet = models.squeezenet1_1(pretrained=True)
        self.features = nn.Sequential(*list(self.squeezenet.children())[:-1])
        self.gap = nn.AdaptiveAvgPool2d((1, 1))

    def forward(self, x):
        x = self.features(x)
        x = self.gap(x)  # Apply Global Average Pooling
        return x.view(x.size(0), -1) 
        #return self.features(x).view(x.size(0), -1)

In [17]:
# Move the model to CPU
model = FeatureExtractor().eval().to(device)

c:\Users\soura\anaconda3\envs\pdsv\Lib\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
c:\Users\soura\anaconda3\envs\pdsv\Lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=SqueezeNet1_1_Weights.IMAGENET1K_V1`. You can also use `weights=SqueezeNet1_1_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


In [ ]:
class calculate_projection:
    def __init__(self, data_base, cnn_model, input_folder, features_folder):
        self.data_base = data_base
        self.cnn_model = cnn_model
        self.input_folder = input_folder
        self.features_folder = features_folder
        self.preprocess = transforms.Compose([
                            transforms.ToPILImage(),
                            transforms.Resize((224, 224)),
                            transforms.ToTensor(),
                            transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
                        ])


    def generate_projections(self, stl_file_path):
        try:
            # Load STL file with numpy-stl
            your_mesh = np_mesh.Mesh.from_file(stl_file_path)
            print(f"Loaded {stl_file_path}")  # Debug print

            # Extract vertices and faces
            vertices = your_mesh.vectors.reshape(-1, 3)
            faces = np.arange(len(vertices)).reshape(-1, 3)

            ranges = vertices.max(axis=0)-vertices.min(axis=0)
            middle = (vertices.max(axis=0)+vertices.min(axis=0))/2
            largest_diff, largest_diff_ind = np.max(ranges), np.argmax(ranges)
            eps = 0.05

            X_min, X_max = 0, 0
            Y_min, Y_max = 0, 0 
            Z_min, Z_max = 0, 0
            if(largest_diff_ind == 0):
                X_min, X_max = middle[0]-largest_diff/2-10*eps, middle[0]+largest_diff/2+10*eps
                Y_min, Y_max = middle[1]-largest_diff/2-eps, middle[1]+largest_diff/2+eps
                Z_min, Z_max = middle[2]-largest_diff/2-eps, middle[2]+largest_diff/2+eps

            if(largest_diff_ind == 1):
                X_min, X_max = middle[0]-largest_diff/2-eps, middle[0]+largest_diff/2+eps
                Y_min, Y_max = middle[1]-largest_diff/2-10*eps, middle[1]+largest_diff/2+10*eps
                Z_min, Z_max = middle[2]-largest_diff/2-eps, middle[2]+largest_diff/2+eps
                
            if(largest_diff_ind == 2):
                X_min, X_max = middle[0]-largest_diff/2-eps, middle[0]+largest_diff/2+eps
                Y_min, Y_max = middle[1]-largest_diff/2-eps, middle[1]+largest_diff/2+eps
                Z_min, Z_max = middle[2]-largest_diff/2-10*eps, middle[2]+largest_diff/2+10*eps

            # Create a figure for plotting
            fig = plt.figure(figsize=(4, 4))

            # Isometric view
            iso_views = []
            for ele, azi in [(30, 45), (30, 225), (-30, 135), (-30, 315)]:
                ax = fig.add_subplot(111, projection='3d')
                ax.plot_trisurf(vertices[:, 0], vertices[:, 1], vertices[:, 2], triangles=faces, cmap='gray', edgecolor='none')
                ax.view_init(elev=ele, azim=azi)
                ax.set_xlim(X_min, X_max)
                ax.set_ylim(Y_min, Y_max)
                ax.set_zlim(Z_min, Z_max)
                plt.axis('off')
                fig.tight_layout(pad=0)
                fig.canvas.draw()
                iso_view = np.frombuffer(fig.canvas.tostring_rgb(), dtype=np.uint8).reshape(fig.canvas.get_width_height()[::-1] + (3,))
                iso_views.append(iso_view)
                plt.clf()

            # Orthographic views
            ortho_views = []
            for ele, azi in [(90, 0), (-90, 0), (0, 0), (0, 90), (0, 180), (0, 270)]:
                ax = fig.add_subplot(111, projection='3d')
                ax.plot_trisurf(vertices[:, 0], vertices[:, 1], vertices[:, 2], triangles=faces, cmap='gray', edgecolor='none')
                ax.view_init(elev=ele, azim=azi)
                ax.set_xlim(X_min, X_max)
                ax.set_ylim(Y_min, Y_max)
                ax.set_zlim(Z_min, Z_max)
                plt.axis('off')
                fig.tight_layout(pad=0)
                fig.canvas.draw()
                ortho_view = np.frombuffer(fig.canvas.tostring_rgb(), dtype=np.uint8).reshape(fig.canvas.get_width_height()[::-1] + (3,))
                ortho_views.append(ortho_view)
                plt.clf()

            plt.close(fig)
            print(f"Generated projections for {stl_file_path}")  # Debug print
            return iso_views, ortho_views
        except Exception as e:
            print(f"Error processing {stl_file_path}: {e}")
            return None, None
        
    def create_batch(self, proj):
        processed_images = []
        for img in proj:
            img_tensor = self.preprocess(img)  # Apply transformations
            processed_images.append(img_tensor)

        # Stack processed images into a single tensor
        batch_tensor = torch.stack(processed_images)
        return batch_tensor
    
    def process_files(self, stl_files):
        for i, stl_file in enumerate(stl_files, 1):  # Start enumeration at 1
            file_path = os.path.join(self.input_folder, stl_file)

            # Generate projections
            iso_views, ortho_views = self.generate_projections(file_path)

            if iso_views is None or ortho_views is None:
                continue # Skip if there was an error in generating projections

            # Extract features from projections
            projections = iso_views + ortho_views
            batch_tensor = self.create_batch(projections)
            proj_feature = 0
            with torch.no_grad():
                proj_feature = self.cnn_model(batch_tensor.to(device)).cpu().numpy().flatten()

            # Save the features with the same name as the STL file
            feature_file_name = os.path.join(self.features_folder, f"{os.path.splitext(stl_file)[0]}_features.npy")
            np.save(feature_file_name, proj_feature)
            self.data_base.send_feature_vector_to_db(stl_file, feature_file_name)
            # Print progress
            print(f"Processed and saved features for {i}/{len(stl_files)}: {stl_file}")

In [19]:
calculate_projection_object = calculate_projection(s1, model, input_folder, proj_features_folder)

In [20]:
calculate_projection_object.process_files(stl_files)

Loaded C:\Users\soura\Desktop\PDSV_codes\stliitm\1013826_1-323326_prt.stl


C:\Users\soura\AppData\Local\Temp\ipykernel_8740\2362165419.py:63: MatplotlibDeprecationWarning: The tostring_rgb function was deprecated in Matplotlib 3.8 and will be removed in 3.10. Use buffer_rgba instead.
  iso_view = np.frombuffer(fig.canvas.tostring_rgb(), dtype=np.uint8).reshape(fig.canvas.get_width_height()[::-1] + (3,))
C:\Users\soura\AppData\Local\Temp\ipykernel_8740\2362165419.py:79: MatplotlibDeprecationWarning: The tostring_rgb function was deprecated in Matplotlib 3.8 and will be removed in 3.10. Use buffer_rgba instead.
  ortho_view = np.frombuffer(fig.canvas.tostring_rgb(), dtype=np.uint8).reshape(fig.canvas.get_width_height()[::-1] + (3,))


Generated projections for C:\Users\soura\Desktop\PDSV_codes\stliitm\1013826_1-323326_prt.stl
The vectors are entered in the database
Processed and saved features for 1/4266: 1013826_1-323326_prt.stl
Loaded C:\Users\soura\Desktop\PDSV_codes\stliitm\1013858_1-643172_prt.stl
Generated projections for C:\Users\soura\Desktop\PDSV_codes\stliitm\1013858_1-643172_prt.stl
The vectors are entered in the database
Processed and saved features for 2/4266: 1013858_1-643172_prt.stl
Loaded C:\Users\soura\Desktop\PDSV_codes\stliitm\1038615_116-1054_prt.stl
Generated projections for C:\Users\soura\Desktop\PDSV_codes\stliitm\1038615_116-1054_prt.stl
The vectors are entered in the database
Processed and saved features for 3/4266: 1038615_116-1054_prt.stl
Loaded C:\Users\soura\Desktop\PDSV_codes\stliitm\1040428_0000000317p2_prt.stl
Generated projections for C:\Users\soura\Desktop\PDSV_codes\stliitm\1040428_0000000317p2_prt.stl
The vectors are entered in the database
Processed and saved features for 4/4266:

# For testing purpose

In [35]:
vector_representation_path, flag = s1.find_feature_vector(stl_files[5])
vector_representation_path

'C:\\Users\\soura\\Desktop\\PDSV_codes\\stl_5000_feature_folder\\proj_features\\1040433_p2_prt_features.npy'

In [36]:
vector_representation_path = vector_representation_path.replace("\\", "/")
vector_representation_path

'C:/Users/soura/Desktop/PDSV_codes/stl_5000_feature_folder/proj_features/1040433_p2_prt_features.npy'

In [37]:
if os.path.exists(vector_representation_path):
    print(f"The directory is : {vector_representation_path}")
else:
    print(f"The directory does not exist")

The directory is : C:/Users/soura/Desktop/PDSV_codes/stl_5000_feature_folder/proj_features/1040433_p2_prt_features.npy


In [38]:
vector_representative = np.load(vector_representation_path)
vector = np.array(vector_representative)
print(vector.shape)

(5120,)


In [39]:
file_name, vector_representation_path, flag = s1.find_feature_vector_through_id(6)
print(file_name)
vector_representation_path

1040433_p2_prt.stl


'C:\\Users\\soura\\Desktop\\PDSV_codes\\stl_5000_feature_folder\\proj_features\\1040433_p2_prt_features.npy'

# For normal feature creation

In [21]:
class Normal_Surface_feature_generation:
    def __init__(self, Normal_Surface_data_base, input_folder, normal_features_folder, surface_features_folder):
        self.Normal_Surface_data_base = Normal_Surface_data_base
        self.input_folder = input_folder
        self.normal_features_folder = normal_features_folder
        self.surface_features_folder = surface_features_folder
        
    def calculate_direction_vector(self, angle_x, angle_y):
        angle_x = np.radians(angle_x)
        angle_y = np.radians(angle_y)
        
        x = np.cos(angle_x) * np.cos(angle_y)
        y = np.cos(angle_x) * np.sin(angle_y)
        z = np.sin(angle_x)
        
        return np.array([x, y, z]) / np.linalg.norm([x, y, z])
    
    def ray_triangle_intersect(self, origin, direction, vertices, epsilon=1e-6):
        # Triangle vertices
        v0, v1, v2 = vertices
        
        # Edge vectors of the triangle
        edge1 = v1 - v0
        edge2 = v2 - v0
        
        # Calculate the determinant
        h = np.cross(direction, edge2)
        a = np.dot(edge1, h)
        
        # If the determinant is near zero, the ray is parallel to the triangle
        if abs(a) < epsilon:
            return False

        f = 1.0 / a
        s = origin - v0
        u = f * np.dot(s, h)
        
        # Check if u is within bounds
        if u < 0.0 or u > 1.0:
            return False

        q = np.cross(s, edge1)
        v = f * np.dot(direction, q)
        
        # Check if v is within bounds and u + v <= 1
        if v < 0.0 or u + v > 1.0:
            return False

        # Calculate t to find the intersection point
        t = f * np.dot(edge2, q)
        
        # Check if the intersection is valid (t > 0 indicates intersection)
        if t > epsilon:
            return True  # There is a valid intersection
        
        return False  # No valid intersection

    
    def find_intersecting_facet(self, mesh, direction_vector):
        center = np.mean(mesh.points.reshape(-1, 3), axis=0)
        for i, triangle in enumerate(mesh.vectors):
            normal = mesh.normals[i]
            vertices = triangle
            if self.ray_triangle_intersect(center, direction_vector, vertices):
                return normal, vertices
        return None, None

    def find_normal(self, stl_file_path):
        # Load the STL file
        try:
            your_mesh = np_mesh.Mesh.from_file(stl_file_path)
            lst = [(30, 45), (30, 225), (-30, 135), (-30, 315), (90, 0), (-90, 0), (0, 0), (0, 90), (0, 180), (0, 270)]
            
            vertices = your_mesh.vectors.reshape(-1, 3)
            volume, cog, inertia = your_mesh.get_mass_properties()

            ranges = vertices.max(axis=0)-vertices.min(axis=0)

            # Surface Area
            surface_area = 0.5 * np.linalg.norm(
                np.cross(your_mesh.vectors[:, 1] - your_mesh.vectors[:, 0], your_mesh.vectors[:, 2] - your_mesh.vectors[:, 0]), axis=1).sum()

            vector = np.array([ranges[2], ranges[0], ranges[1], surface_area, volume])

            # Define the direction vector
            concatenated_vector = np.array([0])
            for ele in lst:
                direction = self.calculate_direction_vector(ele[0],  ele[1])

                # Find the intersecting facet and its normal
                normal, vertices = self.find_intersecting_facet(your_mesh, direction)

                if normal is None:
                    normal = np.array([0, 0, 0])

                concatenated_vector = np.concatenate([concatenated_vector, normal])
            
            return concatenated_vector, vector
        except Exception as e:
            print(f"Error processing {stl_file_path}: {e}")
            return None, None
        
    def process_files(self, stl_files):
        for i, stl_file in enumerate(stl_files, 1):  # Start enumeration at 1
            file_path = os.path.join(self.input_folder, stl_file)

            normal_vector, surface_vector = self.find_normal(file_path)
            if normal_vector is None or surface_vector is None:
                continue

            # Save the normal features with the same name as the STL file
            normal_feature_file = os.path.join(self.normal_features_folder, f"{os.path.splitext(stl_file)[0]}_normal_features.npy")
            np.save(normal_feature_file, normal_vector)

            # Save the normal features with the same name as the STL file
            surface_feature_file = os.path.join(self.surface_features_folder, f"{os.path.splitext(stl_file)[0]}_surface_features.npy")
            np.save(surface_feature_file, surface_vector)

            #Save the features with the same name as the STL file
            self.Normal_Surface_data_base.send_feature_vector_to_db(stl_file,
                                                                     normal_feature_file,
                                                                     surface_feature_file)
            # Print progress
            print(f"Processed and saved features for {i}/{len(stl_files)}: {stl_file}")


In [22]:
Normal_Surface_feature_generation_instance = Normal_Surface_feature_generation(n1,
                                                                                input_folder,
                                                                                normal_features_folder,
                                                                                surface_features_folder)

In [ ]:
Normal_Surface_feature_generation_instance.process_files(stl_files)

# For testing purpose

In [67]:
normal_path, surface_path,  flag = n1.find_feature_vector(stl_files[5])
normal_path

'C:\\Users\\soura\\Desktop\\PDSV_codes\\stl_5000_feature_folder\\normal_features\\1040433_p2_prt_normal_features.npy'

In [68]:
surface_path

'C:\\Users\\soura\\Desktop\\PDSV_codes\\stl_5000_feature_folder\\surface_features\\1040433_p2_prt_surface_features.npy'

In [69]:
normal_path = normal_path.replace("\\", "/")
normal_path

'C:/Users/soura/Desktop/PDSV_codes/stl_5000_feature_folder/normal_features/1040433_p2_prt_normal_features.npy'

In [70]:
surface_path = surface_path.replace("\\", "/")
surface_path

'C:/Users/soura/Desktop/PDSV_codes/stl_5000_feature_folder/surface_features/1040433_p2_prt_surface_features.npy'

In [71]:
if os.path.exists(normal_path):
    print(f"The directory is : {normal_path}")
    normal_vactor = np.load(normal_path)
    normal_vactor = np.array(normal_vactor)
    print(normal_vactor.shape)
else:
    print(f"The directory does not exist")

The directory is : C:/Users/soura/Desktop/PDSV_codes/stl_5000_feature_folder/normal_features/1040433_p2_prt_normal_features.npy
(31,)


In [72]:
if os.path.exists(surface_path):
    print(f"The directory is : {surface_path}")
    surface_vactor = np.load(surface_path)
    surface_vactor = np.array(surface_vactor)
    print(surface_vactor.shape)
else:
    print(f"The directory does not exist")

The directory is : C:/Users/soura/Desktop/PDSV_codes/stl_5000_feature_folder/surface_features/1040433_p2_prt_surface_features.npy
(5,)
